In [1]:
import json
from pprint import pprint

with open("response.json") as f:
    data = json.load(f)

pprint(data["root"][2].keys())
    
# for s in (data["root"][2]['X'][0]):
#     if isinstance(s, str) and ":" in s:
#         print(s) 

dict_keys(['info', 'X', 'X0', 'X00', 'X01', 'X1', 'X10', 'X11', '#f'])


In [2]:
speakers = set()
dialogue = []
for choice_keys in data["root"][2].keys():
    if not choice_keys.startswith("X"):
        continue
    for s in (data["root"][2]['X'][0]):
        if isinstance(s, str) and ":" in s:
            speaker, speech = [l.strip() for l in s.split(":")]
            speaker = speaker.replace("^", "")
            speech = speech[(speech.find(')') + 1):]
            speakers.add(speaker)
            dialogue.append((speaker, speech, str(len(dialogue))))
            # print(speaker, "<=>", "<=>", speech) 
dialogue

[('Charlie',
  "This Enchanted Meadow is so beautiful, isn't it, glowing ball?",
  '0'),
 ('Glowing ball',
  'Yes, it is. The colorful flowers and gentle breeze make it a perfect place for a walk.',
  '1'),
 ('Charlie',
  'Wait, did you hear that rustling sound coming from behind that bush?',
  '2'),
 ('Glowing ball', "Yes, I did. Let's go check it out.", '3'),
 ('Charlie',
  "Oh, it's just a small, mischievous squirrel. Hey there, little guy. What are you up to?",
  '4'),
 ('Rustle', "I'm just playing a little game. Want to join me?", '5'),
 ('Charlie',
  'This game is so much fun! I love exploring and discovering hidden treasures.',
  '6'),
 ('Charlie', 'Did you hear that, glowing ball?', '7'),
 ('Glowing ball',
  'Yes, I did. It sounds like someone is in trouble. We should investigate.',
  '8'),
 ('Charlie',
  "Oh no, Flutter the butterfly is crying. What's wrong, Flutter?",
  '9'),
 ('Flutter',
  "I lost my favorite flower. It's a special flower that only grows in the Enchanted Mea

In [11]:
import json

def save_to_json(data, output_filename):
    """
    Convert a list of tuples to JSON and save to a file.

    Parameters:
    data (list): List of tuples to convert.
    output_filename (str): Filename to save the JSON data.
    """
    # Transform the list of tuples into a list of dictionaries
    json_data = [
        {"speaker": d["speaker"], "speech": d["speech"], "filename": d["filename"]}
        for d in data
    ]
    print(json_data)

    # Serialize and save to a file
    with open(output_filename, 'w') as json_file:
        json.dump(json_data, json_file, indent=4)

# save_to_json(dialogue, "response.json")

In [12]:
import json

with open("response.json") as f:
    dialogue = json.load(f)
with open("response1.json") as f:
    dialogue2 = json.load(f)

save_to_json(dialogue + dialogue2[len(dialogue):], "responsex.json")

[{'speaker': 'Charlie', 'speech': "This Enchanted Meadow is so beautiful, isn't it, glowing ball?", 'filename': '0'}, {'speaker': 'Glowing ball', 'speech': 'Yes, it is. The colorful flowers and gentle breeze make it a perfect place for a walk.', 'filename': '1'}, {'speaker': 'Charlie', 'speech': 'Wait, did you hear that rustling sound coming from behind that bush?', 'filename': '2'}, {'speaker': 'Glowing ball', 'speech': "Yes, I did. Let's go check it out.", 'filename': '3'}, {'speaker': 'Charlie', 'speech': "Oh, it's just a small, mischievous squirrel. Hey there, little guy. What are you up to?", 'filename': '4'}, {'speaker': 'Rustle', 'speech': "I'm just playing a little game. Want to join me?", 'filename': '5'}, {'speaker': 'Charlie', 'speech': 'This game is so much fun! I love exploring and discovering hidden treasures.', 'filename': '6'}, {'speaker': 'Charlie', 'speech': 'Did you hear that, glowing ball?', 'filename': '7'}, {'speaker': 'Glowing ball', 'speech': 'Yes, I did. It sou

In [7]:

import torch
torch.manual_seed(0)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

import random
random.seed(0)

import numpy as np
np.random.seed(0)

# load packages
import time
import random
import yaml
from munch import Munch
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torchaudio
import librosa
from nltk.tokenize import word_tokenize

from models import *
from utils import *
from text_utils import TextCleaner
textclenaer = TextCleaner()


177


In [10]:
to_mel = torchaudio.transforms.MelSpectrogram(
    n_mels=80, n_fft=2048, win_length=1200, hop_length=300)
mean, std = -4, 4

def length_to_mask(lengths):
    mask = torch.arange(lengths.max()).unsqueeze(0).expand(lengths.shape[0], -1).type_as(lengths)
    mask = torch.gt(mask+1, lengths.unsqueeze(1))
    return mask

def preprocess(wave):
    wave_tensor = torch.from_numpy(wave).float()
    mel_tensor = to_mel(wave_tensor)
    mel_tensor = (torch.log(1e-5 + mel_tensor.unsqueeze(0)) - mean) / std
    return mel_tensor

def compute_style(path):
    wave, sr = librosa.load(path, sr=24000)
    audio, index = librosa.effects.trim(wave, top_db=30)
    if sr != 24000:
        audio = librosa.resample(audio, sr, 24000)
    mel_tensor = preprocess(audio).to(device)

    with torch.no_grad():
        ref_s = model.style_encoder(mel_tensor.unsqueeze(1))
        ref_p = model.predictor_encoder(mel_tensor.unsqueeze(1))

    return torch.cat([ref_s, ref_p], dim=1)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# load phonemizer
import phonemizer
global_phonemizer = phonemizer.backend.EspeakBackend(language='en-us', preserve_punctuation=True,  with_stress=True)

config = yaml.safe_load(open("Models/LibriTTS/config.yml"))

# load pretrained ASR model
ASR_config = config.get('ASR_config', False)
ASR_path = config.get('ASR_path', False)
text_aligner = load_ASR_models(ASR_path, ASR_config)

# load pretrained F0 model
F0_path = config.get('F0_path', False)
pitch_extractor = load_F0_models(F0_path)

# load BERT model
from Utils.PLBERT.util import load_plbert
BERT_path = config.get('PLBERT_dir', False)
plbert = load_plbert(BERT_path)


model_params = recursive_munch(config['model_params'])
model = build_model(model_params, text_aligner, pitch_extractor, plbert)
_ = [model[key].eval() for key in model]
_ = [model[key].to(device) for key in model]

params_whole = torch.load("Models/LibriTTS/epochs_2nd_00020.pth", map_location='cpu')
params = params_whole['net']


for key in model:
    if key in params:
        print('%s loaded' % key)
        try:
            model[key].load_state_dict(params[key])
        except:
            from collections import OrderedDict
            state_dict = params[key]
            new_state_dict = OrderedDict()
            for k, v in state_dict.items():
                name = k[7:] # remove `module.`
                new_state_dict[name] = v
            # load params
            model[key].load_state_dict(new_state_dict, strict=False)
#             except:
#                 _load(params[key], model[key])
_ = [model[key].eval() for key in model]

from Modules.diffusion.sampler import DiffusionSampler, ADPM2Sampler, KarrasSchedule

sampler = DiffusionSampler(
    model.diffusion.diffusion,
    sampler=ADPM2Sampler(),
    sigma_schedule=KarrasSchedule(sigma_min=0.0001, sigma_max=3.0, rho=9.0), # empirical parameters
    clamp=False
)

def inference(text, ref_s, alpha = 0.3, beta = 0.7, diffusion_steps=5, embedding_scale=1):
    text = text.strip()
    ps = global_phonemizer.phonemize([text])
    ps = word_tokenize(ps[0])
    ps = ' '.join(ps)
    tokens = textclenaer(ps)
    tokens.insert(0, 0)
    tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)
    
    with torch.no_grad():
        input_lengths = torch.LongTensor([tokens.shape[-1]]).to(device)
        text_mask = length_to_mask(input_lengths).to(device)

        t_en = model.text_encoder(tokens, input_lengths, text_mask)
        bert_dur = model.bert(tokens, attention_mask=(~text_mask).int())
        d_en = model.bert_encoder(bert_dur).transpose(-1, -2) 

        s_pred = sampler(noise = torch.randn((1, 256)).unsqueeze(1).to(device), 
                                          embedding=bert_dur,
                                          embedding_scale=embedding_scale,
                                            features=ref_s, # reference from the same speaker as the embedding
                                             num_steps=diffusion_steps).squeeze(1)


        s = s_pred[:, 128:]
        ref = s_pred[:, :128]

        ref = alpha * ref + (1 - alpha)  * ref_s[:, :128]
        s = beta * s + (1 - beta)  * ref_s[:, 128:]

        d = model.predictor.text_encoder(d_en, 
                                         s, input_lengths, text_mask)

        x, _ = model.predictor.lstm(d)
        duration = model.predictor.duration_proj(x)

        duration = torch.sigmoid(duration).sum(axis=-1)
        pred_dur = torch.round(duration.squeeze()).clamp(min=1)


        pred_aln_trg = torch.zeros(input_lengths, int(pred_dur.sum().data))
        c_frame = 0
        for i in range(pred_aln_trg.size(0)):
            pred_aln_trg[i, c_frame:c_frame + int(pred_dur[i].data)] = 1
            c_frame += int(pred_dur[i].data)

        # encode prosody
        en = (d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device))
        if model_params.decoder.type == "hifigan":
            asr_new = torch.zeros_like(en)
            asr_new[:, :, 0] = en[:, :, 0]
            asr_new[:, :, 1:] = en[:, :, 0:-1]
            en = asr_new

        F0_pred, N_pred = model.predictor.F0Ntrain(en, s)

        asr = (t_en @ pred_aln_trg.unsqueeze(0).to(device))
        if model_params.decoder.type == "hifigan":
            asr_new = torch.zeros_like(asr)
            asr_new[:, :, 0] = asr[:, :, 0]
            asr_new[:, :, 1:] = asr[:, :, 0:-1]
            asr = asr_new

        out = model.decoder(asr, 
                                F0_pred, N_pred, ref.squeeze().unsqueeze(0))
    
        
    return out.squeeze().cpu().numpy()[..., :-50] # weird pulse at the end of the model, need to be fixed later

/home/gpuserver/miniconda3/envs/for_whisper/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/home/gpuserver/miniconda3/envs/for_whisper/lib/python3.10/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


bert loaded
bert_encoder loaded
predictor loaded
decoder loaded
text_encoder loaded
predictor_encoder loaded
style_encoder loaded
diffusion loaded
text_aligner loaded
pitch_extractor loaded
mpd loaded
msd loaded
wd loaded


In [17]:
import numpy as np
import io
from pydub import AudioSegment

def numpy_array_to_mp3(audio_array, out_path, frame_rate=24000):
    """
    Convert a 1D NumPy array to MP3 format in-memory.

    Parameters:
    audio_array (numpy.ndarray): The 1D numpy array representing audio data.
    frame_rate (int): The sampling rate of the audio data.

    Returns:
    bytes: The MP3 formatted audio data.
    """

    # Normalize and convert to 16-bit PCM
    audio_array = (audio_array * 32767).astype(np.int16)

    # Create a BytesIO object to save the WAV file in-memory
    wav_buffer = io.BytesIO()
    wav_buffer.write(audio_array.tobytes())

    # Rewind the buffer to the beginning
    wav_buffer.seek(0)

    # Read the WAV using pydub
    audio_segment = AudioSegment.from_raw(wav_buffer, sample_width=2, frame_rate=frame_rate, channels=1)


    # Export as MP3 to the in-memory buffer
    audio_segment.export(out_path, format="mp3")

In [19]:
from tqdm.auto import tqdm

start = time.time()
noise = torch.randn(1,1,256).to(device)
voice_ref_dict = {
    'Charlie' : "Demo/reference_audio/Vinay.wav", 
    'Flutter': "Demo/reference_audio/1221-135767-0014.wav", 
    'Glowing Ball': "Demo/reference_audio/696_92939_000016_000006.wav", 
    'Glowing ball': "Demo/reference_audio/696_92939_000016_000006.wav", 
    'Rustle':"Demo/reference_audio/908-157963-0027.wav"
}

ref_dict = {k:compute_style(v) for k, v in voice_ref_dict.items()}

for speaker, speech, fname in tqdm(dialogue):
    wav = inference(speech, ref_dict[speaker], alpha=0.3, beta=0.7, diffusion_steps=10, embedding_scale=2)
    numpy_array_to_mp3(wav, out_path=f"magic_speech/{fname}.mp3")
    # print(wav.shape)
    # break

# ref_path = ""
#     ref_s = compute_style(path)
    
#     wav = inference(text, ref_s, alpha=0.3, beta=0.7, diffusion_steps=5, embedding_scale=1)
#     rtf = (time.time() - start) / (len(wav) / 24000)
#     print(f"RTF = {rtf:5f}")
#     import IPython.display as ipd
#     print(k + ' Synthesized:')
#     display(ipd.Audio(wav, rate=24000, normalize=False))
#     print('Reference:')
#     display(ipd.Audio(path, rate=24000, normalize=False))


100%|██████████| 147/147 [00:26<00:00,  5.56it/s]


In [9]:
import json

input_file_path = 'test.ink'  # Replace with your input file path
output_file_path = 'output.jsonl'  # Replace with your desired output file path

def process_line(line_number, line):
    speaker, speech = line.split(':', 1)
    return {
        "lineno": line_number,
        "speaker_name": speaker.strip(),
        "Speech": speech.strip()
    }

with open(input_file_path, 'r') as input_file, open(output_file_path, 'w') as output_file:
    for lineno, line in enumerate(input_file, start=1):
        #  not line.startswith("\t") and
        if ':' in line and "" not in line.strip().split(':', 1):
            print(line.split(':', 1))
            json_line = process_line(lineno, line)
            json.dump(json_line, output_file)
            output_file.write('\n')


['enchanted meadow ', ' The Enchanted Meadow is a serene and peaceful place filled with colorful flowers, butterflies, and fireflies. The meadow is surrounded by tall trees that provide shade and shelter to the animals that live there.\n']
['Charlie', " This Enchanted Meadow is so beautiful, isn't it, glowing ball?\n"]
['Glowing ball', ' Yes, it is. The colorful flowers and gentle breeze make it a perfect place for a walk.\n']
['Charlie', " This Enchanted Meadow is so beautiful. I'm glad we decided to explore it.\n"]
['Charlie', " Look, Glowing Ball, there's a group of animals having a picnic over there. Let's go join them!\n"]
['Animal', ' Welcome, Charlie and Glowing Ball! Please join us for some delicious food.\n']
['Charlie', " Thank you, this food is amazing! It's so nice to have some company on our journey.\n"]
['Charlie', " Hey, Glowing Ball, what's wrong? Why are you moving away from me?\n"]
['Glowing Ball', " I don't know, Charlie. I just feel like we're growing apart. We used

In [3]:
import json

output_file_path = 'output.jsonl'  # Replace with your JSONL file path

def read_jsonl_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            json_line = json.loads(line.strip())
            data.append(json_line)
    return data

# Call the function and store the list of dictionaries
list_of_dicts = read_jsonl_file(output_file_path)

# Print the list of dictionaries
for item in list_of_dicts:
    print(item)


{'lineno': 6, 'speaker_name': 'enchanted meadow', 'Speech': 'The Enchanted Meadow is a serene and peaceful place filled with colorful flowers, butterflies, and fireflies. The meadow is surrounded by tall trees that provide shade and shelter to the animals that live there.'}
{'lineno': 10, 'speaker_name': 'Charlie', 'Speech': 'Charlie is a scruffy brown dog with a heart of gold. He has a keen sense of smell and hearing, which makes him a valuable asset in the jungle full of ghosts. He is always ready to help other animals in need, even if it means putting himself in danger. Charlie is a loyal friend and a great listener. He has a playful personality and loves to run around and explore the jungle. He is also very curious and always eager to learn new things.'}
{'lineno': 11, 'speaker_name': 'Glowing Ball', 'Speech': 'The glowing ball is a mysterious entity that emits a soft, pulsating light. It is small enough to fit in the palm of your hand and has a smooth, glassy surface. It has a fri

In [7]:
import pandas as pd

# # Assuming 'list_of_dicts' is your list of dictionaries
# list_of_dicts = [{'lineno': 1, 'speaker_name': 'Speaker1', 'Speech': 'Hello world'},
#                  {'lineno': 2, 'speaker_name': 'Speaker2', 'Speech': 'Another example'}]

# Convert to DataFrame
df = pd.DataFrame(list_of_dicts)

# Print the DataFrame
df['speaker_name'] = df['speaker_name'].apply(lambda x: x.lower())
df

,lineno,speaker_name,Speech
0,6,enchanted meadow,The Enchanted Meadow is a serene and peaceful ...
1,10,charlie,Charlie is a scruffy brown dog with a heart of...
2,11,glowing ball,The glowing ball is a mysterious entity that e...
3,14,pounce,Pounce is a nimble leopard who helps Harlie cl...
4,15,flutter,Flutter is a colorful butterfly who loves to e...
...,...,...,...
109,435,charlie,Do you know where we can find the blackstone s...
110,437,glowing ball,"Yes, I do. But it's not going to be easy. We'l..."
111,447,charlie,"We don't want to fight you, Claw. But we will ..."
112,449,claw,(roars and lunges at Charlie)


In [8]:
df['speaker_name'].value_counts()

charlie             68
glowing ball        24
claw                 8
pounce               4
spike                3
enchanted meadow     1
flutter              1
splash               1
rustle               1
animal               1
jungle spirit        1
animals              1
Name: speaker_name, dtype: int64

In [2]:
import json

input_file_path = 'DialogOutput.txt'  # Replace with your input file path
output_file_path = 'output.jsonl'  # Replace with your desired output file path

def process_line(line_number, line):
    speaker, speech = line.split(':', 1)
    return {
        "lineno": line_number,
        "speaker_name": speaker.strip(),
        "Speech": speech.strip()
    }

with open(input_file_path, 'r') as input_file, open(output_file_path, 'w') as output_file:
    for lineno, line in enumerate(input_file, start=1):
        #  not line.startswith("\t") and
        if ':' in line and "" not in line.strip().split(':', 1):
            speaker, speech = line.strip().split(':', 1)
            speech = speech.strip()
            if not  speech.startswith("(") and not speech.endswith(")"):
                print(line)
            # print(line.split(':', 1))
            # json_line = process_line(lineno, line)
            # json.dump(json_line, output_file)
            # output_file.write('\n')


Charlie: Wait, did you hear that rustling sound coming from behind that bush?

Charlie: Wait, did you hear that rustling sound coming from behind that bush?

Glowing ball: Yes, I did. Let's go check it out.

Glowing ball: Yes, I did. Let's go check it out.

Charlie: Oh, it's just a small, mischievous squirrel. Hey there, little guy. What are you up to?

Charlie: Oh, it's just a small, mischievous squirrel. Hey there, little guy. What are you up to?

Rustle: I'm just playing a little game. Want to join me?

Rustle: I'm just playing a little game. Want to join me?

Charlie: This game is so much fun! I love exploring and discovering hidden treasures.

Charlie: This game is so much fun! I love exploring and discovering hidden treasures.

Charlie: Did you hear that, glowing ball?

Charlie: Did you hear that, glowing ball?

Glowing ball: Yes, I did. It sounds like someone is in trouble. We should investigate.

Glowing ball: Yes, I did. It sounds like someone is in trouble. We should investig